In [ ]:
#Smit Patel


# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#New Version with all of the data added to input
import os
os.chdir('/kaggle/input/alldata/Project')

In [ ]:
!pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

In [ ]:
!pip install Cython
!pip install git+https://github.com/philferriere/cocoapi.git#egg=pycocotools^&subdirectory=PythonAPI

In [ ]:
os.chdir('detr')

In [ ]:
#DETR Imports
!pwd

import argparse
import random
import cv2
from pathlib import Path
import torch
import torchvision.transforms as T
import PIL.Image
from models import build_model
from main import get_args_parser
import sys
import json
import shutil
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Layer

from models_lfw.face_recognition.model_small import create_model
from models_lfw.face_recognition.align import AlignDlib
from models_lfw.triplet_loss import TripletLossLayer
from coml_preprocessor import ComlFaceDataGenerator

In [ ]:
##WE NEED TO ADD IN A DATSET FOR THE 
import csv
mapping = {}
with open('/content/drive/Shareddrives/2021 FIRE-COML-STUDENTS/Spring/Facial Recognition Team Project/Face Photos Dataset/person_name_id_mapping.csv', newline='') as csvfile:
     reader = csv.DictReader(csvfile)
     for row in reader:
         print(row["Name"], int(row["ID"]))
         mapping[row["Name"]] = int(row["ID"]) # This stores Id in a key-value pair format similar to a java map

In [ ]:
# Model Configurations
parser = argparse.ArgumentParser(description='DETR args parser', parents=[get_args_parser()])
args = parser.parse_args(args=[])
args.resume = '/content/drive/Shareddrives/2021 FIRE-COML-STUDENTS/Spring/Facial Recognition Team Project/Team 3/Project/models_lfw/checkpoint.pth'
args.device = 'cpu'

if args.output_dir:
    Path(args.output_dir).mkdir(parents=True, exist_ok=True)

args.distributed = False

print(args) 

# Buuld Model
model, criterion, postprocessors = build_model(args)

device = torch.device(args.device)
model.to(device)

# Load Weights
output_dir = Path(args.output_dir)
if args.resume:
    # the model will download the weights and model state from the https link provided
    if args.resume.startswith('https'):
      checkpoint = torch.hub.load_state_dict_from_url(
          args.resume, map_location='cpu', check_hash = True)
    else:
        checkpoint = torch.load(args.resume, map_location='cpu')

    # this load the weights and model state into the model
    model.load_state_dict(checkpoint['model'], strict=True)

In [ ]:
# COCO classes
CLASSES = [
   'N/A', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
   'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A',
   'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse',
   'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack',
   'umbrella', 'N/A', 'N/A', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis',
   'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
   'skateboard', 'surfboard', 'tennis racket', 'bottle', 'N/A', 'wine glass',
   'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich',
   'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake',
   'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table', 'N/A',
   'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
   'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A',
   'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier',
   'toothbrush'
]

# colors for visualization
COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]

# standard PyTorch mean-std input image normalization
transform = T.Compose([
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# for output bounding box post-processing
def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)

def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return b

def detect(im, model, transform):
    # mean-std normalize the input image (batch-size: 1)
    img = transform(im).unsqueeze(0)

    assert img.shape[-2] <= 1600 and img.shape[-1] <=1600, 'demo model only supports images up to 1600 pixels on each side'

    outputs = model(img)

    probas = outputs['pred_logits'].softmax(-1)[0, :, :-1]
    keep = probas.max(-1).values > 0.7

    bboxes_scaled = rescale_bboxes(outputs['pred_boxes'][0, keep], im.size)
    return probas[keep], bboxes_scaled

def plot_results(pil_img, prob, boxes, classes):
    plt.figure(figsize=(16,10))
    plt.imshow(pil_img)
    ax = plt.gca()
    for p, (xmin, ymin, xmax, ymax), c in zip(prob, boxes.tolist(), COLORS * 100):
        c1 = p.argmax()
        if CLASSES[c1] not in classes:
            continue
        ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                   fill=False, color=c, linewidth=3))
        
        text= f'Persons name'
        #f'{CLASSES[c1]}: {p[c1]:0.2f}'
        ax.text(xmin, ymin, text, fontsize=15,
                bbox=dict(facecolor='yellow', alpha=.5))
    plt.axis('off')
    plt.show()

# Stores coordinates of the bounding box
def bbox_dims(pil_img, prob, boxes, classes):
    bbox_list = []
    for p, (xmin, ymin, xmax, ymax), c in zip(prob, boxes.tolist(), COLORS * 100):
        c1 = p.argmax()
        if CLASSES[c1] not in classes:
            continue
        bbox_list.append([xmin, ymin, xmax, ymax])
    return bbox_list

In [ ]:
the_image = PIL.Image.open('/content/a.jpg')

width, height = the_image.size

if width > 1600 and height > 1600:
  new_size = (1600, 1600)
  the_image = the_image.resize(new_size)

#resized_im = the_image.resize((round(the_image.size[0]*0.5), round(the_image.size[1]*0.5)))
scores, boxes = detect(the_image, model, transform)
plot_classes = ['person']
plot_results(the_image, scores, boxes, plot_classes)

# Obtains bbox coordinates for each bbox and adds them to the list
bbox_dim_list = bbox_dims(the_image, scores, boxes, plot_classes)

#for bbox in bbox_dim_list:
#  print(bbox)

In [ ]:
# Crop Images, this will be fed into the coml generator
cropped_images = []
for bbox in bbox_dim_list:
    cropped_images.append(the_image.crop((bbox[0], bbox[1], bbox[2], bbox[3]))) # Left, upper, right, bottom
print(cropped_images)

i = 1
for bbox in bbox_dim_list:
  plt.subplot(1, len(cropped_images), i)
  plt.imshow(cropped_images[i - 1])
  i += 1
plt.show()

LFW

In [ ]:
# Input for anchor, positive, and negative images
in_a = Input(shape=(96, 96, 3), name="img_a")
in_p = Input(shape=(96, 96, 3), name="img_p")
in_n = Input(shape=(96, 96, 3), name="img_n")

# create the base model from model_small
model_sm = create_model()

# Output the embedding vectors from anchor, positive, and negative images
# The model weights are shared (Triplet network)
emb_a = model_sm(in_a)
emb_p = model_sm(in_p)
emb_n = model_sm(in_n)

# Layer that computes the triplet loss from anchor, positive and negative embedding vectors
triplet_loss_layer = TripletLossLayer(alpha=0.2, name='triplet_loss_layer')([emb_a, emb_p, emb_n])

# Model that can be trained with anchor, positive, and negative images
model = Model([in_a, in_p, in_n], triplet_loss_layer)

# Load weights
model.load_weights("/content/drive/Shareddrives/2021 FIRE-COML-STUDENTS/Spring/Facial Recognition Team Project/Team 3/Project/ckpts(lfw)/epoch024_loss1.440.hdf5")

base_model = model.layers[3]

In [ ]:
class IdentityMetadata():
    def __init__(self, base, name, file):
        # dataset base directory
        self.base = base
        # identity name
        self.name = name
        # image file name
        self.file = file

    def __repr__(self):
        return self.image_path()

    def image_path(self):
        return os.path.join(self.base, self.name, self.file) 
    
def load_metadata(path, upper_limit):
    metadata = []
    count = 0
    for i in sorted(os.listdir(path)):
        if count == upper_limit: 
            break
            
        for f in sorted(os.listdir(os.path.join(path, i))):
            if count == upper_limit: 
                break
                
            count += 1
            # Check file extension. Allow only jpg/jpeg' files.
            ext = os.path.splitext(f)[1]
            if ext == '.jpg' or ext == '.jpeg':
                metadata.append(IdentityMetadata(path, i, f))
    return np.array(metadata)

def load_image(path):
    img = cv2.imread(path, 1)
    # OpenCV loads images with color channels
    # in BGR order. So we need to reverse them
    return img[...,::-1]
    
def align_image(img):
        alignment = AlignDlib('models_lfw/landmarks.dat')
        bb = alignment.getLargestFaceBoundingBox(img)
        if bb is None:
            return cv2.resize(img, (96,96))
        else:
            return alignment.align(96, 
                                   img, 
                                   bb,
                                   landmarkIndices=AlignDlib.OUTER_EYES_AND_NOSE)

In [ ]:
metadata = load_metadata('/content/drive/Shareddrives/2021 FIRE-COML-STUDENTS/Spring/Facial Recognition Team Project/Team 3/Project/data/fire', 100)
alignment = AlignDlib('models_lfw/landmarks.dat')

In [ ]:
myEmbeddings = np.empty((metadata.shape[0], 128))

for i,img in enumerate(cropped_images):
  img = align_image(np.asarray(img))
  img = img.astype('float32')
  img =img / 255.0
  img = np.expand_dims(img, axis=0)
  person = str(i)#m.image_path().split('/')[11].replace('_', ' ')
  # base_model.summary()
  myEmbeddings[i] = base_model.predict(img)
    # for displaying the progress of creating embeddings
  if i%1 == 0: print(str(i) + "\n", end=" ")

In [ ]:
embeddings = np.empty((metadata.shape[0], 128))

for i, m in enumerate(metadata):
    img = load_image(m.image_path())
    img = align_image(img)
    img = img.astype('float32')
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    person = m.image_path().split('/')[11].replace('_', ' ')
   # base_model.summary()
    embeddings[i] = base_model.predict(img)
    
    # for displaying the progress of creating embeddings
    if i%1 == 0: print(str(i) + " " + person + "\n", end=" ")
    if i == len(metadata)-1: print('done')

In [ ]:
def distance(emb1, emb2):
    return np.sum(np.square(emb1 - emb2))

In [ ]:
person = {}
def show_pair_m(myEmbeddings, embeddings, idMy, idE):
    plt.figure(figsize=(4,2))
    plt.suptitle(f'Distance = {distance(myEmbeddings[idMy], embeddings[idE]):.2f}') 

    plt.subplot(1,2,1)
    plt.imshow(align_image(np.asarray(cropped_images[idMy])))
    plt.subplot(1,2,2)
    plt.imshow(align_image(load_image(metadata[idE].image_path())))

    if (idMy == 0):
      person[idMy] = [metadata[idE].image_path().split('/')[11].replace('_', ' ')]
    if (idMy != 0):
      dist = distance(myEmbeddings[idMy], embeddings[idE])
      if dist < distance(myEmbeddings[idMy - 1], embeddings[idE - 1]):
        #p.append(metadata[idE].image_path().split('/')[11].replace('_', ' '))
        personidMy = [metadata[idE].image_path().split('/')[11].replace('_', ' ')]

    plt.show()
    
# show positive pair (change this to what you want)

for i in range(len(myEmbeddings)):
  for b in range(len(embeddings)):
    show_pair_m(myEmbeddings, embeddings, i, b)

print(person)

#def show_pair_m(myEmbeddings, embeddings, idMy, idE):
 #   plt.figure(figsize=(4,2))
  #  plt.suptitle(f'Distance = {distance(myEmbeddings[idMy], embeddings[idE]):.2f}') 
#
 #   plt.subplot(1,2,1)
  #  plt.imshow(align_image(np.asarray(cropped_images[idMy])))
   # plt.subplot(1,2,2)
    #plt.imshow(align_image(load_image(metadata[idE].image_path())))

    #plt.show()
    
# show positive pair (change this to what you want)
#show_pair_m(myEmbeddings, embeddings, 10, 2)

In [ ]:
# Obtain ID Numbers
for i in range(len(person)):
  print(mapping[str(person[i]).replace("[", "").replace("]", "").replace("'", "")])

In [ ]:
def plot_results2(pil_img, prob, boxes, classes):
    plt.figure(figsize=(16,10))
    plt.imshow(pil_img)
    ax = plt.gca()
    for p, (xmin, ymin, xmax, ymax), c in zip(prob, boxes.tolist(), COLORS * 100):
        c1 = p.argmax()
        if CLASSES[c1] not in classes:
            continue
        ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                   fill=False, color=c, linewidth=3))
        
        
        for i in range(len(person)):
          text = str(person[i]).replace("[", "").replace("]", "").replace("'", "")
        


        #f'{CLASSES[c1]}: {p[c1]:0.2f}'
        ax.text(xmin, ymin, text, fontsize=15,
                bbox=dict(facecolor='yellow', alpha=.5))
    plt.axis('off')
    plt.show()

In [ ]:
plot_results2(the_image, scores, boxes, plot_classes)

In [ ]:
persons = []
path = []

def show_pair(embeddings, id1,secondemb, id2):
    plt.figure(figsize=(4,2))
    plt.suptitle(f'Distance = {distance(embeddings[id1], embeddings[id2]):.2f}')

    #imagePath = str(metadata[id2].image_path()) # Stores ImagePath, idea is to use ID2 as the name of the person it could be
    #path.append(imagePath)

    plt.subplot(1,2,1)
    plt.imshow(align_image(load_image(metadata[id1].image_path())))
    plt.subplot(1,2,2)
    "person " + str(id2)
    plt.imshow(cropped_images[id2])
    plt.imshow(align_image(load_image(metadata[id2].image_path())))

    dist = distance(embeddings[id1], embeddings[id2])
    if dist < .5:
        persons.append(metadata[id1].image_path().split('/')[11].replace('_', ' '))
        persons.append(metadata[id2].image_path().split('/')[11].replace('_', ' '))
    plt.show()
    
# show positive pair (change this to what you want)
show_pair(embeddings, 3, 9)

# show negative pair (change this to what you want)
#show_pair(embeddings, 3, 9)

print(persons)
#print(imagePath)
print(path)

In [ ]:
import tensorflow as tf
from tensorflow import keras

class ComlFaceDataGenerator(keras.utils.Sequence):
    def __init__(self, data_dir, batch_size, anchor_shape = (96,96), n_channels = 3, img_aug = False, face_align = True, shuffle = True):
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.img_aug = img_aug
        self.anchor_shape = anchor_shape
        self.n_channels = n_channels
        self.img_aug = img_aug
        self.shuffle = shuffle
        self.face_align = face_align
        self.alignment = AlignDlib('models_lfw/landmarks.dat')
        
        persons = os.listdir(data_dir)
        img_exts = ('.png', '.jpg', '.jpeg')

        # process pos and neg image pairs from data_dir
        self.positive_pairs = []
        self.negative_pairs = []
        for i, person in enumerate(persons):
            person_path = os.path.join(data_dir, person)
            person_files = os.listdir(person_path)
            # get all image files based on extension
            image_files = [image_file for image_file in person_files if image_file.endswith(img_exts)]
            
            # randomly pick 2 images of same person as positive pair 
            if len(image_files) >= 2:
                image_ids = np.arange(len(image_files))
                random.shuffle(image_ids)
                self.positive_pairs.append([(person,image_files[image_ids[0]]),
                                            (person,image_files[image_ids[1]])])
                
                # randomly pick another image as negative pair
                remaining_persons = persons.copy()
                remaining_persons.pop(i)
                while True:
                    other_person = random.choice(remaining_persons)
                    other_person_path = os.path.join(data_dir, other_person)
                    other_person_files = os.listdir(other_person_path)
                    other_person_image_files = [image_file for image_file in other_person_files if image_file.endswith(img_exts)]
                    if len(other_person_image_files) >= 1:
                        random_id = random.randrange(len(other_person_image_files))
                        self.negative_pairs.append([(person,image_files[image_ids[0]]),
                                                    (other_person,other_person_image_files[random_id])])
                        break
            else:
                continue

        self.on_epoch_end()

    # update indexs after each epoch
    def on_epoch_end(self):
        self.pos_indexes = np.arange(len(self.positive_pairs))
        self.neg_indexes = np.arange(len(self.negative_pairs))
        if self.shuffle == True:
            np.random.shuffle(self.pos_indexes)
            np.random.shuffle(self.neg_indexes)

    # return the number of batches per epoch
    def __len__(self):        
        return int(np.floor(len(self.pos_indexes) / self.batch_size))

    # return the path to the image
    def get_image_path(self, person, image_file):
        return os.path.join(self.data_dir, person, image_file)

    # augment img as a batch
    def augment_images(self, img_batch):
        seq = iaa.Sequential([
            iaa.Crop(px=(0, 16)), # crop images from each side by 0 to 16px (randomly chosen)
            iaa.Fliplr(0.5), # horizontally flip 50% of the images
            iaa.GaussianBlur(sigma=(0, 1.0)),
            # Strengthen or weaken the contrast in each image.
            iaa.LinearContrast((0.75, 1.5)),# blur images with a sigma of 0 to 3.0
        ])
        return seq(images=img_batch)

    def align_image(self, img):
        alignment = self.alignment
        bb = alignment.getLargestFaceBoundingBox(img)
        if bb is None:
            return cv2.resize(img, self.anchor_shape)
        else:
            return alignment.align(self.anchor_shape[0],
                                   img,
                                   bb,
                                   landmarkIndices=AlignDlib.OUTER_EYES_AND_NOSE)
    
    # return the image as numpy tensor
    def get_image_tensor(self, person, image_file):
        # read and remove alpha channel
        img = cv2.imread(self.get_image_path(person, image_file))[:,:,:3] 
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if self.face_align:
            img = self.align_image(img)
        else:
            img = cv2.resize(img, self.anchor_shape)
        img = img / 255.0
        img = img.astype(np.float32)
        img = img[None, ...]
        return img

    # feed in an index and return a batch
    def __getitem__(self, index):
        # slice indexes for current batch
        pos_indexes = self.pos_indexes[index*self.batch_size:(index+1)*self.batch_size]
        neg_indexes = self.neg_indexes[index*self.batch_size:(index+1)*self.batch_size]
        
        anchor_img_arr = np.empty( (self.batch_size, *self.anchor_shape, self.n_channels), dtype=np.float32 )
        pos_img_arr = np.empty( (self.batch_size, *self.anchor_shape, self.n_channels), dtype=np.float32 )
        neg_img_arr = np.empty( (self.batch_size, *self.anchor_shape, self.n_channels), dtype=np.float32 )

        pos_pairs_batch = [self.positive_pairs[j] for j in pos_indexes]
        neg_pairs_batch = [self.negative_pairs[k] for k in neg_indexes]

        for i, pos_pair in enumerate(pos_pairs_batch):
            # process anchor image
            anc_name, anc_id = pos_pair[0]
            anchor_img_arr[i] = self.get_image_tensor(anc_name, anc_id)

            # process postive image
            pos_name, pos_id = pos_pair[1]
            pos_img_arr[i] = self.get_image_tensor(pos_name, pos_id)
            
            # Process negative image
            neg_pair = neg_pairs_batch[i]
            neg_name, neg_id = neg_pair[0]
            if pos_name == neg_name:
                neg_name, neg_id = neg_pair[1]
            neg_img_arr[i] = self.get_image_tensor(neg_name, neg_id)

        if self.img_aug:
            anchor_img_arr = self.augment_images(anchor_img_arr)
            pos_img_arr = self.augment_images(pos_img_arr)
            neg_img_arr = self.augment_images(neg_img_arr)

        return anchor_img_arr, pos_img_arr, neg_img_arr

In [ ]:
#from coml_preprocessor import ComlFaceDataGenerator
data_dir = '/content/drive/Shareddrives/2021 FIRE-COML-STUDENTS/Spring/Facial Recognition Team Project/Team 3/Project/data/fire'
batch_size = 5
coml_generator = ComlFaceDataGenerator(data_dir=data_dir,
                                       batch_size=batch_size,
                                       img_aug=False)

pos_pairs_embeddings = np.empty((len(coml_generator.positive_pairs), 2, 128))
pos_images = {}

for i, pair in enumerate(coml_generator.positive_pairs):
    person, image_file = pair[0]
    img = coml_generator.get_image_tensor(person, image_file)
    pos_pairs_embeddings[i, 0] = base_model.predict(img)

    person, image_file = pair[1]
    img1 = coml_generator.get_image_tensor(person, image_file)
    pos_pairs_embeddings[i, 1] = base_model.predict(img1)
    
    pos_images[i] = [img[0,:,:,:], img1[0,:,:,:]]

    if i%1 == 0: print(i, end=" ")
    if i == len(coml_generator.positive_pairs)-1: print('done')

In [ ]:
import tensorflow as tf
from tensorflow import keras

class ComlFaceDataGenerator(keras.utils.Sequence):
    def __init__(self, data_dir, batch_size, anchor_shape = (96,96), n_channels = 3, img_aug = False, face_align = True, shuffle = True):
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.img_aug = img_aug
        self.anchor_shape = anchor_shape
        self.n_channels = n_channels
        self.img_aug = img_aug
        self.shuffle = shuffle
        self.face_align = face_align
        self.alignment = AlignDlib('models_lfw/landmarks.dat')
        
        persons = os.listdir(data_dir)
        img_exts = ('.png', '.jpg', '.jpeg')

        # process pos and neg image pairs from data_dir
        self.positive_pairs = []
        self.negative_pairs = []
        for i, person in enumerate(persons):
            person_path = os.path.join(data_dir, person)
            person_files = os.listdir(person_path)
            # get all image files based on extension
            image_files = [image_file for image_file in person_files if image_file.endswith(img_exts)]
            
            # randomly pick 2 images of same person as positive pair 
            if len(image_files) >= 2:
                image_ids = np.arange(len(image_files))
                random.shuffle(image_ids)
                self.positive_pairs.append([(person,image_files[image_ids[0]]),
                                            (person,image_files[image_ids[1]])])
                
                # randomly pick another image as negative pair
                remaining_persons = persons.copy()
                remaining_persons.pop(i)
                while True:
                    other_person = random.choice(remaining_persons)
                    other_person_path = os.path.join(data_dir, other_person)
                    other_person_files = os.listdir(other_person_path)
                    other_person_image_files = [image_file for image_file in other_person_files if image_file.endswith(img_exts)]
                    if len(other_person_image_files) >= 1:
                        random_id = random.randrange(len(other_person_image_files))
                        self.negative_pairs.append([(person,image_files[image_ids[0]]),
                                                    (other_person,other_person_image_files[random_id])])
                        break
            else:
                continue

        self.on_epoch_end()

    # update indexs after each epoch
    def on_epoch_end(self):
        self.pos_indexes = np.arange(len(self.positive_pairs))
        self.neg_indexes = np.arange(len(self.negative_pairs))
        if self.shuffle == True:
            np.random.shuffle(self.pos_indexes)
            np.random.shuffle(self.neg_indexes)

    # return the number of batches per epoch
    def __len__(self):        
        return int(np.floor(len(self.pos_indexes) / self.batch_size))

    # return the path to the image
    def get_image_path(self, person, image_file):
        return os.path.join(self.data_dir, person, image_file)

    # augment img as a batch
    def augment_images(self, img_batch):
        seq = iaa.Sequential([
            iaa.Crop(px=(0, 16)), # crop images from each side by 0 to 16px (randomly chosen)
            iaa.Fliplr(0.5), # horizontally flip 50% of the images
            iaa.GaussianBlur(sigma=(0, 1.0)),
            # Strengthen or weaken the contrast in each image.
            iaa.LinearContrast((0.75, 1.5)),# blur images with a sigma of 0 to 3.0
        ])
        return seq(images=img_batch)

    def align_image(self, img):
        alignment = self.alignment
        bb = alignment.getLargestFaceBoundingBox(img)
        if bb is None:
            return cv2.resize(img, self.anchor_shape)
        else:
            return alignment.align(self.anchor_shape[0],
                                   img,
                                   bb,
                                   landmarkIndices=AlignDlib.OUTER_EYES_AND_NOSE)
    
    # return the image as numpy tensor
    def get_image_tensor(self, person, image_file):
        # read and remove alpha channel
        img = cv2.imread(self.get_image_path(person, image_file))[:,:,:3] 
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if self.face_align:
            img = self.align_image(img)
        else:
            img = cv2.resize(img, self.anchor_shape)
        img = img / 255.0
        img = img.astype(np.float32)
        img = img[None, ...]
        return img

    # feed in an index and return a batch
    def __getitem__(self, index):
        # slice indexes for current batch
        pos_indexes = self.pos_indexes[index*self.batch_size:(index+1)*self.batch_size]
        neg_indexes = self.neg_indexes[index*self.batch_size:(index+1)*self.batch_size]
        
        anchor_img_arr = np.empty( (self.batch_size, *self.anchor_shape, self.n_channels), dtype=np.float32 )
        pos_img_arr = np.empty( (self.batch_size, *self.anchor_shape, self.n_channels), dtype=np.float32 )
        neg_img_arr = np.empty( (self.batch_size, *self.anchor_shape, self.n_channels), dtype=np.float32 )

        pos_pairs_batch = [self.positive_pairs[j] for j in pos_indexes]
        neg_pairs_batch = [self.negative_pairs[k] for k in neg_indexes]

        for i, pos_pair in enumerate(pos_pairs_batch):
            # process anchor image
            anc_name, anc_id = pos_pair[0]
            anchor_img_arr[i] = self.get_image_tensor(anc_name, anc_id)

            # process postive image
            pos_name, pos_id = pos_pair[1]
            pos_img_arr[i] = self.get_image_tensor(pos_name, pos_id)
            
            # Process negative image
            neg_pair = neg_pairs_batch[i]
            neg_name, neg_id = neg_pair[0]
            if pos_name == neg_name:
                neg_name, neg_id = neg_pair[1]
            neg_img_arr[i] = self.get_image_tensor(neg_name, neg_id)

        if self.img_aug:
            anchor_img_arr = self.augment_images(anchor_img_arr)
            pos_img_arr = self.augment_images(pos_img_arr)
            neg_img_arr = self.augment_images(neg_img_arr)

        return anchor_img_arr, pos_img_arr, neg_img_arr

In [ ]:
#from coml_preprocessor import ComlFaceDataGenerator
data_dir = '/content/drive/Shareddrives/2021 FIRE-COML-STUDENTS/Spring/Facial Recognition Team Project/Team 3/Project/data/fire'
batch_size = 5
coml_generator = ComlFaceDataGenerator(data_dir=data_dir,
                                       batch_size=batch_size,
                                       img_aug=False)

pos_pairs_embeddings = np.empty((len(coml_generator.positive_pairs), 2, 128))
pos_images = {}

for i, pair in enumerate(coml_generator.positive_pairs):
    person, image_file = pair[0]
    img = coml_generator.get_image_tensor(person, image_file)
    pos_pairs_embeddings[i, 0] = base_model.predict(img)

    person, image_file = pair[1]
    img1 = coml_generator.get_image_tensor(person, image_file)
    pos_pairs_embeddings[i, 1] = base_model.predict(img1)
    
    pos_images[i] = [img[0,:,:,:], img1[0,:,:,:]]

    if i%1 == 0: print(i, end=" ")
    if i == len(coml_generator.positive_pairs)-1: print('done')

In [ ]:
def distance(emb1, emb2):
    return np.sum(np.square(emb1 - emb2))

def show_pair_dist(pair_embeddings, images, id):
    plt.figure(figsize=(4,2))
    plt.suptitle(f'Distance = {distance(pair_embeddings[id, 0], pair_embeddings[id, 1]):.2f}')
    plt.subplot(121)
    plt.imshow(images[id][0])
    plt.subplot(122)
    plt.imshow(images[id][1])
    plt.show()
show_pair_dist(pos_pairs_embeddings, pos_images, 2)